<a href="https://colab.research.google.com/github/reva3b/2d-platformer/blob/master/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Imports / Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [3]:
import os
import numpy as np
import random
import pandas as pd
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
print('Tensorflow Version:', tf.__version__)

Tensorflow Version: 2.0.0


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, Conv2DTranspose
from skimage.color import rgb2lab, lab2rgb, rgb2gray

In [5]:
# change current working directory
os.chdir('/content/drive/My Drive/CISC 6000: Deep Learning/')
os.getcwd()

'/content/drive/My Drive/CISC 6000: Deep Learning'

In [0]:
# seed for randoom number generator
SEED = 6000
def seed_everything(SEED):
    np.random.seed(SEED)
    tf.random.set_seed(SEED)
    random.seed(SEED)
seed_everything(SEED)

# Prepare Data for Training

In [0]:
# returns batches of X, Y data for given generator (train/val)
def getBatches(generator):
    for batch in generator:
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [0]:
BATCH_SIZE = 8
TARGET_SIZE = (128, 128)

In [22]:
# specify image data generators
#, shear_range=0.2, zoom_range=0.2, rotation_range=20, horizontal_flip=True)
train_datagen = ImageDataGenerator(data_format='channels_last', rescale=1./255, validation_split=0.235)
train_generator = train_datagen.flow_from_directory(directory='./Final Project (Deep Learning - Fall 2019)/DATA/Flowers', 
                                                    subset='training', classes=['train'], class_mode=None, seed=SEED,
                                                    target_size=TARGET_SIZE, batch_size=BATCH_SIZE)
val_generator = train_datagen.flow_from_directory(directory='./Final Project (Deep Learning - Fall 2019)/DATA/Flowers',
                                                  subset='validation', classes=['train'], class_mode=None, seed=SEED,
                                                  target_size=TARGET_SIZE, batch_size=BATCH_SIZE)

Found 100 images belonging to 1 classes.
Found 30 images belonging to 1 classes.


In [24]:
# check size/scale for train and val images
x = train_generator[0]
print('Shape of TRAIN Batch:', x[0].shape, 'with images in range', x[0].min(), 'to', x[0].max())
x = val_generator[0]
print('Shape of VAL Batch:', x[0].shape, 'with images in range', x[0].min(), 'to', x[0].max())

Shape of TRAIN Batch: (128, 128, 3) with images in range 0.0 to 0.9960785
Shape of VAL Batch: (128, 128, 3) with images in range 0.0 to 1.0


# Baseline Model

In [0]:
def baselineModel(input_shape, starting_filter, num_layers):
    model = Sequential()
    model.add(InputLayer(input_shape=input_shape, name='input'))
    for i in [2**j for j in range(num_layers)]:
        model.add(Conv2D(filters=starting_filter * i, kernel_size=(3, 3), activation='relu', 
                         padding='same', strides=1, name='conv_' + str(starting_filter * i)))
        model.add(Conv2D(filters=starting_filter * i, kernel_size=(3, 3), activation='relu', 
                         padding='same', strides=2, name='stridePool_' + str(starting_filter * i)))
    for i in [2**j for j in range(num_layers - 1, -1, -1)]:
        model.add(Conv2DTranspose(filters=starting_filter * i, kernel_size=(3, 3), activation='relu', 
                                  padding='same', strides=2, name='convT_' + str(starting_filter * i)))
    model.add(Conv2D(filters=2, kernel_size=(3, 3), activation='tanh', 
                     padding='same', strides=1, name='output'))
    return model

In [86]:
baseline = baselineModel(input_shape=TARGET_SIZE+(1,), starting_filter=32, num_layers=3)
baseline.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_32 (Conv2D)             (None, 128, 128, 32)      320       
_________________________________________________________________
stridePool_32 (Conv2D)       (None, 64, 64, 32)        9248      
_________________________________________________________________
conv_64 (Conv2D)             (None, 64, 64, 64)        18496     
_________________________________________________________________
stridePool_64 (Conv2D)       (None, 32, 32, 64)        36928     
_________________________________________________________________
conv_128 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
stridePool_128 (Conv2D)      (None, 16, 16, 128)       147584    
_________________________________________________________________
convT_128 (Conv2DTranspose)  (None, 32, 32, 128)     